## Install the package dependencies before running this notebook

In [4]:
import torch
from torch.utils.data import Dataset, DataLoader
import os, os.path 
import numpy 
import pickle
from glob import glob

"""
    number of trajectories in each city
    # austin --  train: 43041 test: 6325 
    # miami -- train: 55029 test:7971
    # pittsburgh -- train: 43544 test: 6361
    # dearborn -- train: 24465 test: 3671
    # washington-dc -- train: 25744 test: 3829
    # palo-alto -- train:  11993 test:1686

    trajectories sampled at 10HZ rate, input 5 seconds, output 6 seconds
    
"""

'\n    number of trajectories in each city\n    # austin --  train: 43041 test: 6325 \n    # miami -- train: 55029 test:7971\n    # pittsburgh -- train: 43544 test: 6361\n    # dearborn -- train: 24465 test: 3671\n    # washington-dc -- train: 25744 test: 3829\n    # palo-alto -- train:  11993 test:1686\n\n    trajectories sampled at 10HZ rate, input 5 seconds, output 6 seconds\n    \n'

## Create a Torch.Dataset class for the training dataset

In [5]:
from glob import glob
import pickle
import numpy as np

ROOT_PATH = "./"

cities = ["austin", "miami", "pittsburgh", "dearborn", "washington-dc", "palo-alto"]
splits = ["train", "test"]

def get_city_trajectories(city="palo-alto", split="train", normalized=False):

    
    outputs = None
    
    if split=="train":
        f_in = ROOT_PATH + split + "/" + city + "_inputs"
        inputs = pickle.load(open(f_in, "rb"))
        n = len(inputs)
        inputs = np.asarray(inputs)[:int(n * 0.8)]
        
        f_out = ROOT_PATH + split + "/" + city + "_outputs"
        outputs = pickle.load(open(f_out, "rb"))
        outputs = np.asarray(outputs)[:int(n * 0.8)]
        
    elif split == 'val':
        f_in = ROOT_PATH + 'train' + "/" + city + "_inputs"
        inputs = pickle.load(open(f_in, "rb"))
        n = len(inputs)
        inputs = np.asarray(inputs)[int(n * 0.8):]
        
        f_out = ROOT_PATH + 'train' + "/" + city + "_outputs"
        outputs = pickle.load(open(f_out, "rb"))
        outputs = np.asarray(outputs)[int(n * 0.8):]
    
    else:
        f_in = ROOT_PATH + split + "/" + city + "_inputs"
        inputs = pickle.load(open(f_in, "rb"))
        n = len(inputs)
        inputs = np.asarray(inputs)

    return inputs, outputs

class ArgoverseDataset(Dataset):
    """Dataset class for Argoverse"""
    def __init__(self, city: str, split:str, transform=None):
        super(ArgoverseDataset, self).__init__()
        self.transform = transform

        self.inputs, self.outputs = get_city_trajectories(city=city, split=split, normalized=False)

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):

        data = (self.inputs[idx], self.outputs[idx])
            
        if self.transform:
            data = self.transform(data)

        return data

# intialize a dataset
city = 'palo-alto' 
split = 'train'
train_dataset  = ArgoverseDataset(city = city, split = split)

In [6]:
val_dataset = ArgoverseDataset(city = city, split = 'val')

In [7]:
len(val_dataset)

2399

## Linear Model

In [48]:
batch_sz = 4  # batch size 
train_loader = DataLoader(train_dataset,batch_size=batch_sz)

In [49]:
from torch import nn, optim

class Pred(nn.Module):

    def __init__(self):
        super().__init__()
        
        self.encoder = nn.Sequential(
            nn.Linear(100, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 32)
        )
        
        self.decoder = nn.Sequential(
            nn.Linear(32, 64),
            nn.ReLU(),
            nn.Linear(64, 120),
            nn.ReLU(),
            nn.Linear(120, 120)
        )
        
    def forward(self, x):
        x = x.reshape(-1, 100).float()
        x = self.encoder(x)
        x = self.decoder(x)
        x = x.reshape(-1, 60, 2)
        return x

In [50]:
pred = Pred()
opt = optim.Adam(pred.parameters(), lr=1e-3)

In [51]:
for epoch in range(50):
    
    total_loss = 0
    for i_batch, sample_batch in enumerate(train_loader):
        inp, out = sample_batch
        preds = pred(inp)
        loss = ((preds - out) ** 2).sum()
        
        opt.zero_grad()
        loss.backward()
        opt.step()
        
        total_loss += loss.item()
        
    print('epoch {} loss: {}'.format(epoch, total_loss / len(train_dataset)))

epoch 0 loss: 5297945.011629367
epoch 1 loss: 982669.6537442857
epoch 2 loss: 830850.6337423348
epoch 3 loss: 657516.8131961257
epoch 4 loss: 633075.8472834211
epoch 5 loss: 534921.5247922165
epoch 6 loss: 520223.20100468775
epoch 7 loss: 436538.0449405525
epoch 8 loss: 425720.60604857263
epoch 9 loss: 390754.60499953764
epoch 10 loss: 344878.1286765895
epoch 11 loss: 343042.4085844876
epoch 12 loss: 337213.3357682958
epoch 13 loss: 297765.41315710574
epoch 14 loss: 284382.2233212258
epoch 15 loss: 273616.19813444954
epoch 16 loss: 262412.4611020876
epoch 17 loss: 234371.0990897969
epoch 18 loss: 246920.5017016828
epoch 19 loss: 217747.26117072196
epoch 20 loss: 213540.66577598432
epoch 21 loss: 200993.88239040368
epoch 22 loss: 203805.6637853246
epoch 23 loss: 191141.00862017635
epoch 24 loss: 178153.1603302524
epoch 25 loss: 164941.09705620812
epoch 26 loss: 174452.17771637382
epoch 27 loss: 168355.42085954495
epoch 28 loss: 166833.5988152815
epoch 29 loss: 160584.2063363659
epoch 30

In [52]:
val_loader = DataLoader(val_dataset,batch_size=batch_sz)

val_loss = 0
for i_batch, sample_batch in enumerate(val_loader):
    inp, out = sample_batch
    preds = pred(inp)
    loss = ((preds - out) ** 2).sum()
    val_loss += loss.item()
    
    #show_sample_batch((inp, preds.detach()))
print('loss: {}'.format(val_loss / len(val_dataset)))

loss: 58330.97971981225


In [53]:
palo_alto_test_dataset = ArgoverseDataset(city = 'palo-alto', split = 'test')
palo_alto_test_loader = DataLoader(palo_alto_test_dataset,batch_size=batch_sz)
len(palo_alto_test_loader)

val_loss = 0
for sample_batch in enumerate(palo_alto_test_loader):
    print(i_batch, sample_batch)

TypeError: 'NoneType' object is not subscriptable

## LSTM Model

In [23]:
batch_sz = 64  # batch size 
train_loader = DataLoader(train_dataset,batch_size=batch_sz)

In [24]:
from torch import nn, optim

class PredLSTM(nn.Module):
    def __init__(self, hidden_layers=64):
        super(PredLSTM, self).__init__()
        self.hidden_layers = hidden_layers
        self.lstm = nn.LSTM(2, hidden_layers, 1, batch_first=True)
#         self.lstm2 = nn.LSTM(2, hidden_layers, 1, batch_first=True)
        self.linear = nn.Linear(hidden_layers, 2)
        
        
    def forward(self, x, future_preds=0):
        outputs = []
        h_t = torch.zeros(1, 64, self.hidden_layers)
        c_t = torch.zeros(1, 64, self.hidden_layers)
        output, (h_t, c_t) = self.lstm(x, (h_t, c_t))
        
        for i in range(60):
            output , (h_t, c_t) = self.lstm(x[:, -1:, :], (h_t, c_t))
            out = self.linear(output)
            outputs.append(out)
            
        
#         for input_t in x.split(1, dim=1):
#             # N, 1
#             output, (h_t, c_t) = self.lstm(input_t, (h_t, c_t)) # initial hidden and cell states
#             #print(output.shape)
#             output = self.linear(output)
#             #print(output.shape)
#             #output = self.linear(h_t2) # output from the last FC layer
#             outputs.append(output)
            
#         for i in range(future_preds):
#             # this only generates future predictions if we pass in future_preds>0
#             # mirrors the code above, using last output/prediction as input
#             output, (h_t, c_t) = self.lstm(output, (h_t, c_t))
#             output = self.linear(output)
#             outputs.append(output)
            
        # transform list to tensor    
        outputs = torch.cat(outputs, dim=1)
        return outputs

In [26]:
model = PredLSTM()
print(model)
criterion = nn.MSELoss()
opt = optim.Adam(model.parameters(), lr=1e-3)
# opt = optim.LBFGS(model.parameters(), lr=0.08)

for epoch in range(10):
    
    total_loss = 0
    for i_batch, sample_batch in enumerate(train_loader):
        inp, out = sample_batch
        if len(inp) < 64:
            break
        
        preds = model(inp.type(torch.float))
        #print(preds.shape)
        #break
        loss = ((preds - out) ** 2).sum()
        
        opt.zero_grad()
        loss.backward()
        opt.step()
        
        total_loss += loss.item()   
    print('epoch {} loss: {}'.format(epoch, total_loss / len(train_dataset)))

PredLSTM(
  (lstm): LSTM(2, 64, batch_first=True)
  (linear): Linear(in_features=64, out_features=2, bias=True)
)
epoch 0 loss: 278705371.12429106
epoch 1 loss: 276939337.1672723
epoch 2 loss: 274205838.7232537
epoch 3 loss: 271366288.10392666
epoch 4 loss: 268691307.9379299
epoch 5 loss: 265938459.9928473
epoch 6 loss: 263254327.3694119
epoch 7 loss: 260826781.32984456
epoch 8 loss: 258343650.90628752
epoch 9 loss: 255812706.48835057


## Sample a batch of data and visualize 

In [10]:
import matplotlib.pyplot as plt
import random


def show_sample_batch(sample_batch):
    """visualize the trajectory for a batch of samples"""
    inp, out = sample_batch
    batch_sz = inp.size(0)
    agent_sz = inp.size(1)
    
    fig, axs = plt.subplots(1,batch_sz, figsize=(15, 3), facecolor='w', edgecolor='k')
    fig.subplots_adjust(hspace = .5, wspace=.001)
    axs = axs.ravel()   
    for i in range(batch_sz):
        axs[i].xaxis.set_ticks([])
        axs[i].yaxis.set_ticks([])
        
        # first two feature dimensions are (x,y) positions
        axs[i].scatter(inp[i,:,0], inp[i,:,1])
        axs[i].scatter(out[i,:,0], out[i,:,1])

        
for i_batch, sample_batch in enumerate(train_loader):
    inp, out = sample_batch
    print(inp.shape, out.shape)
    break
    """
    TODO:
      implement your Deep learning model
      implement training routine
    """
    show_sample_batch(sample_batch)
    break

torch.Size([4, 50, 2]) torch.Size([4, 60, 2])


In [42]:
N = 100 # number of samples
L = 1000 # length of each sample (number of values for each sine wave)
T = 20 # width of the wave
x = np.empty((N,L), np.float32) # instantiate empty array
x[:] = np.arange(L) + np.random.randint(-4*T, 4*T, N).reshape(N,1)
y = np.sin(x/1.0/T).astype(np.float32)
x.shape, y.shape

for i_batch, sample_batch in enumerate(train_loader):
    inp, out = sample_batch
    print(inp.shape, out.shape)
    print(inp.split(1, dim=1)[0].shape)
    break
    """
    TODO:
      implement your Deep learning model
      implement training routine
    """
    show_sample_batch(sample_batch)
    break

torch.Size([4, 50, 2]) torch.Size([4, 60, 2])
torch.Size([4, 1, 2])


In [101]:
torch.randn(128, 20), torch.randn(128, 20).shape

(tensor([[ 2.4738,  0.4687, -1.0604,  ..., -1.3048, -0.2083,  0.0795],
         [ 0.3574, -1.1261, -0.9496,  ...,  0.9317,  0.0569, -0.4361],
         [ 0.9958,  0.5853, -0.6282,  ...,  0.4379, -0.7539,  0.7080],
         ...,
         [-0.5143, -2.2022, -1.0705,  ..., -2.4927,  0.1381, -0.5287],
         [ 0.7938,  1.0560,  0.7956,  ..., -0.0968,  0.9463,  0.1143],
         [-0.6036, -0.2649, -0.7492,  ..., -0.5561, -0.1611, -0.6414]]),
 torch.Size([128, 20]))